In [ ]:
import pandas as pd
import seaborn as sns
import torch
from matplotlib import pyplot as plt

import pyro
import pyro.distributions as dist
import pyro.ops.stats as stats

from rethinking import MAP, coef, extract_samples, link, precis, sim, vcov
sns.set(font_scale=1.25, rc={"figure.figsize": (10, 8)})
%config InlineBackend.figure_format = 'retina'

In [ ]:
import sys
sys.path
# /Users/josephmann/anaconda3/envs/pytorch_exp/lib/python3.7/site-packages/matplotlib-3.1.1-py3.7-nspkg.pth

In [ ]:
import holoviews as hv
from holoviews import streams
import panel as pn
from holoviews.operation.datashader import datashade, rasterize, dynspread
hv.extension('bokeh') # important: doesn't render without it
hv.opts.defaults( hv.opts.Curve(width=800), hv.opts.Points(width=800),hv.opts.Histogram(width=800), hv.opts.Points(size=3))

import numpy as np

In [ ]:
n=300
steps = 32
# create random walk data
pos = torch.empty(n, steps).uniform_(-1, 1).cumsum(1)

# make the first position 0
pos = pos - pos[:,0].unsqueeze(1)

# make curves and points
curves = {i: hv.Curve( (torch.arange(steps), pos[i,:])) for i in range(n)}
scatters = {i: hv.Scatter(curves[i])  for i in range(n)}

walk = hv.NdOverlay(scatters).opts(
    hv.opts.Scatter(alpha=0.1, 
                    selection_color='red', 
                    color='black', 
                    tools=['box_select'], 
                    width=600)) 

walk_lines = hv.NdOverlay(curves).opts(
    hv.opts.Curve(
        alpha=0.05, selection_color='red', 
        hover_alpha=0.5, selection_alpha=0.5, 
        color='black', 
#         tools=['','hover'], 
        width=600)) 

selection = streams.Selection1D(source=walk)

walk_df = walk.dframe()



def selected_lines(index):
    global selected_events 
    selected_events = index
    if (index or len(index)>0):
        line_ids = list(walk_df.iloc[index]['Element'].unique()) 
    else:
        line_ids = []
        return walk * walk_lines
    chosen_lines = walk.select(Element=line_ids).opts( 
        hv.opts.Scatter(color='blue', alpha=0.3, tools=['box_select']), 
        hv.opts.NdOverlay(show_legend=False))#, title=str(line_ids)))
    chosen_dots = walk_lines.select(Element=line_ids).opts( 
        hv.opts.Curve(color='blue', alpha=0.3, tools=['box_select']), 
        hv.opts.NdOverlay(show_legend=False, title=str(len(line_ids))))
    return  chosen_dots * chosen_lines #* walk * walk_lines ## the order of the layout may make a difference

layout =  hv.DynamicMap(selected_lines, streams=[selection])
layout.opts(shared_datasource=True, active_tools=['box_select'])
# datashade(layout).opts(tools=['box_select'], width=800 )
walk * datashade(layout).opts(tools=['box_select'], width=800 )

In [ ]:
selection.index

In [47]:
n=300
steps = 32
# create random walk data
pos = torch.empty(n, steps).uniform_(-1, 1).cumsum(1)

# make the first position 0
pos = pos - pos[:,0].unsqueeze(1)

# make curves and points
curves = {i: hv.Curve( (torch.arange(steps), pos[i,:])) for i in range(n)}
scatters = {i: hv.Scatter(curves[i])  for i in range(n)}

walk = hv.NdOverlay(scatters).opts(
    hv.opts.Scatter(alpha=0.1, 
                    selection_color='red', 
                    color='black', 
                    tools=['box_select'], 
                    width=600)) 

walk_lines = hv.NdOverlay(curves).opts(
    hv.opts.Curve(
        alpha=0.05, selection_color='red', 
        hover_alpha=0.5, selection_alpha=0.5, 
        color='black', 
#         tools=['','hover'], 
        width=600)) 

# selection = streams.Selection1D(source=walk)
box2 = streams.BoundsXY(source=walk, bounds=(0,0,0,0))

walk_df = walk.dframe()
walk_ds = hv.Dataset(walk_df)

def box_cb(bounds):
    global gbounds
    gbounds = bounds
    if bounds:
        x0, y0, x1, y1 = bounds
        sel_elements = walk_ds.select(x=(x0,x1),y=(y0,y1))['Element']
        sel_lines = walk_lines.select(Element=list(sel_elements)).opts( hv.opts.Curve(color='blue',alpha=0.4))
        return walk * sel_lines
    else:
        return walk * NdOverlay(dict())
    

# def selected_lines(index):
#     global selected_events 
#     selected_events = index
#     if (index or len(index)>0):
#         line_ids = list(walk_df.iloc[index]['Element'].unique()) 
#     else:
#         line_ids = []
#         return walk * walk_lines
#     chosen_lines = walk.select(Element=line_ids).opts( 
#         hv.opts.Scatter(color='blue', alpha=0.3), 
#         hv.opts.NdOverlay(show_legend=False))#, title=str(line_ids)))
#     chosen_dots = walk_lines.select(Element=line_ids).opts( 
#         hv.opts.Curve(color='blue', alpha=0.3), 
#         hv.opts.NdOverlay(show_legend=False, title=str(len(line_ids))))
#     return walk * chosen_dots * chosen_lines #* walk * walk_lines ## the order of the layout may make a difference

# layout =  hv.DynamicMap(selected_lines, streams=[selection])
blayout =  hv.DynamicMap(box_cb, streams=[box2])
# layout.opts( active_tools=['box_select'])
(blayout).opts(active_tools=['box_select'], width=800, show_legend=False, )

:DynamicMap   []
   :Overlay
      .NdOverlay.I  :NdOverlay   [Element]
         :Scatter   [x]   (y)
      .NdOverlay.II :NdOverlay   [Element]

In [ ]:
line_ids = list(walk.dframe().iloc[selection.index]['Element'])
walk * walk.select(Element= line_ids) * walk_lines.select(Element= line_ids).opts(hv.opts.Curve(alpha=.2))

In [110]:
n=30
steps = 32
# create random walk data
pos = torch.empty(n, steps).uniform_(-1, 1).cumsum(1)

# make the first position 0
pos = pos - pos[:,0].unsqueeze(1)

# make curves and points
curves = {i: hv.Curve( (torch.arange(steps), pos[i,:])) for i in range(n)}
scatters = {i: hv.Scatter(curves[i])  for i in range(n)}

walk = hv.NdOverlay(scatters).opts(
    hv.opts.Scatter(alpha=0.1, 
                    selection_color='red', 
                    color='black', 
                    tools=['box_select'], 
                    width=600)) 

walk_lines = hv.NdOverlay(curves).opts(
    hv.opts.Curve(
        alpha=0.05, selection_color='red', 
        hover_alpha=0.5, selection_alpha=0.5, 
        color='black', 
#         tools=['','hover'], 
        width=600)) 

# selection = streams.Selection1D(source=walk)
box = streams.BoundsXY(source=walk, bounds=(0,0,0,0))

walk_df = walk.dframe()
walk_ds = hv.Dataset(walk_df)

def box_cb(bounds):
    global gbounds
    gbounds = bounds
    if bounds:
        x0, y0, x1, y1 = bounds
        sel_elements = walk_ds.select(x=(x0,x1),y=(y0,y1))['Element']
        sel_lines = walk_lines.select(Element=list(sel_elements)).opts( hv.opts.Curve(color='blue',alpha=0.4, title=str(sel_elements)), )
        return  sel_lines
    else:
        return  NdOverlay(dict())
    

# def selected_lines(index):
#     global selected_events 
#     selected_events = index
#     if (index or len(index)>0):
#         line_ids = list(walk_df.iloc[index]['Element'].unique()) 
#     else:
#         line_ids = []
#         return walk * walk_lines
#     chosen_lines = walk.select(Element=line_ids).opts( 
#         hv.opts.Scatter(color='blue', alpha=0.3), 
#         hv.opts.NdOverlay(show_legend=False))#, title=str(line_ids)))
#     chosen_dots = walk_lines.select(Element=line_ids).opts( 
#         hv.opts.Curve(color='blue', alpha=0.3), 
#         hv.opts.NdOverlay(show_legend=False, title=str(len(line_ids))))
#     return walk * chosen_dots * chosen_lines #* walk * walk_lines ## the order of the layout may make a difference

# layout =  hv.DynamicMap(selected_lines, streams=[selection])
blayout =  hv.DynamicMap(box_cb, streams=[box])
# layout.opts( active_tools=['box_select'])
walk * (blayout).opts(active_tools=['box_select'], width=800, show_legend=False, )

:DynamicMap   []
   :Overlay
      .NdOverlay.I  :NdOverlay   [Element]
         :Scatter   [x]   (y)
      .NdOverlay.II :NdOverlay   [Element]

In [ ]:
walk[(9,)].opts(color='red',alpha=1.0) * walk_lines[(9,)]

In [ ]:
walk[(9,)].opts(color='red',alpha=1.0) * walk_lines[(9,)]

In [ ]:
walk.data[(9,)].opts(color='red',alpha=1.0) * walk_lines[(9,)]

In [ ]:
walk_lines.select(Element=[0,5])

In [ ]:
selection.contents

In [ ]:
walk.dframe().iloc[selection.index]['Element']

In [ ]:
walk.data

In [ ]:
len(selection.index)

In [ ]:
selection.index

In [ ]:
[walk.data[i] for i in selection.index]

In [ ]:
# select just selects sections of the plot to present, doesn't access data underneath.. 
walk_lines.select(y=(0,10))

In [ ]:
%%opts NdOverlay [legend_limit=0] Curve [tools=['tap']] (line_width=1 )
ndoverlay = hv.NdOverlay({i: hv.Curve(np.arange(10)*i) for i in range(5)})

selection = streams.Selection1D(source=ndoverlay)
dmap = hv.DynamicMap(lambda index: ndoverlay[index] if index else ndoverlay.clone(),
                     kdims=[], streams=[selection])
ndoverlay + dmap

In [ ]:
n=30
steps = 32
# create random walk data
pos = torch.empty(n, steps).uniform_(-1, 1).cumsum(1)

# make the first position 0
pos = pos - pos[:,0].unsqueeze(1)

# make curves and points
curves = {i: hv.Curve( (torch.arange(steps), pos[i,:])) for i in range(n)}#.opts(tools=['box_select','click'])
# scatters = {i: hv.Scatter( (torch.arange(steps), pos[i,:])) for i in range(n)}

# walk = hv.NdOverlay(curves).opts(hv.opts.Curve(alpha=0.1, color='black', tools=['xbox_select'])) 
# walk = hv.NdOverlay(scatters).opts(hv.opts.Scatter(alpha=0.1, selection_color='red', color='black', tools=['box_select',], width=600)) 
walk_lines = hv.NdOverlay(curves).opts(hv.opts.Curve(
    alpha=0.01, selection_color='red', hover_alpha=0.5, selection_alpha=0.5, color='black', tools=['box_select','lasso_select','tap','hover'], width=600)) 
# walk_lines = hv.NdOverlay(curves).opts(hv.opts.Curve(alpha=0.01, hover_alpha=0.5, color='black', tools=['xbox_select','hover'], width=600)) 
# walk * 
walk_lines

## Got pn range sliders working 

In [155]:
    
int_range_slider = pn.widgets.IntRangeSlider(
    name='X range',
    start=0, end=32, value=(30, 31), step=1)

float_range_slider = pn.widgets.RangeSlider(name='Y range', start=-6., end=6, step=0.1, value=(1.57, 1.69))

@pn.depends(int_range_slider.param.value, float_range_slider.param.value)
def box_cb2(x_range=(31,32), y_range=(-0.10,0.) ):
    """
    returns the elements have points in the given ranges
    """
#     x_range=value
    x0, x1  = x_range
    y0, y1  = y_range
    sel_elements = np.unique(walk_ds.select(x=(x0,x1),y=(y0,y1))['Element']).flatten()
    sel_lines = walk_lines.select(Element=list(sel_elements)).opts( hv.opts.Curve(color='blue',alpha=0.4, title=str([sel_elements,x_range])), )
    return sel_lines #hv.Dataset(sel_elements )

# dyn_map = hv.DynamicMap(box_cb2, streams = [int_range_slider.param.value, # have to find a way to change the name of this parameter...
#                            ]) #float_range_slider.param.value

# filt_lines =  filtered_ds.apply(lambda e: curves[list(e.data)])#walk_lines.select(Element=e))
row = pn.Row(int_range_slider, float_range_slider )
col = pn.Column( row, box_cb2)

In [156]:
col

Column
    [0] Row
        [0] IntRangeSlider(end=32, name='X range', value=(30, 31))
        [1] RangeSlider(end=6, name='Y range', start=-6.0, step=0.1, value=(1.57, 1.69))
    [1] ParamFunction(function)

In [87]:
curves[]

:Curve   [x]   (y)

In [75]:
float_range_slider.value

(1.57, 3)

In [68]:
sel_elements = walk_ds.select(x=int_range_slider.value,y=float_range_slider.value)['Element']

In [69]:
sel_lines = walk_lines.select(Element=list(sel_elements))

In [70]:
walk *sel_lines

:Overlay
   .NdOverlay.I  :NdOverlay   [Element]
      :Scatter   [x]   (y)
   .NdOverlay.II :NdOverlay   [Element]
      :Curve   [x]   (y)

In [67]:
int_range_slider.param.value.

In [66]:
sel_elements

array([], dtype=int64)

In [71]:
filtered_ds = walk_ds.apply(box_cb2, x_range=int_range_slider.value,
                            y_range=float_range_slider.value)

NameError: name 'box_cb2' is not defined